In [7]:
import pandas as pd

In [8]:
postulantes_df = pd.read_csv("LISTA_DE_POSTULANTES_1.csv", sep=";", encoding="latin1")

In [9]:
postulantes_df.dtypes

FECHA_CORTE             int64
UUID                   object
ESCUELA                object
MODALIDAD              object
ANNIO_POSTULACION       int64
UBIGEO                  int64
DEPARTAMENTO           object
PROVINCIA              object
DISTRITO               object
UBIGEO_ORIGEN           int64
DEPARTAMENTO_ORIGEN    object
PROVINCIA_ORIGEN       object
DISTRITO_ORIGEN        object
UBIGEO_IE               int64
DEPARTAMENTO_IE        object
PROVINCIA_IE           object
DISTRITO_IE            object
EDAD                    int64
SEXO                   object
PAGO                    int64
IDIOMA                 object
LEE                    object
ESCRIBE                object
HABLA                  object
TIENE_DISCAPACIDAD     object
TIPO_COMUNIDAD         object
COMUNIDAD              object
dtype: object

In [10]:
# Función para limpiar números (quitar comas y convertir a float)
def limpiar_numerico(serie):
    if serie.dtype == 'object':
        # Quitar comas y convertir a numeric
        return pd.to_numeric(serie.astype(str).str.replace(',', ''), errors='coerce')
    return serie

# Aplicar conversiones según el diccionario de datos para postulantes
print("🔄 Convirtiendo tipos de datos para postulantes...")

# Campos numéricos enteros
postulantes_df['FECHA_CORTE'] = limpiar_numerico(postulantes_df['FECHA_CORTE']).astype('Int64')
postulantes_df['ANNIO_POSTULACION'] = limpiar_numerico(postulantes_df['ANNIO_POSTULACION']).astype('Int64')
postulantes_df['UBIGEO'] = limpiar_numerico(postulantes_df['UBIGEO']).astype('Int64')
postulantes_df['UBIGEO_ORIGEN'] = limpiar_numerico(postulantes_df['UBIGEO_ORIGEN']).astype('Int64')
postulantes_df['UBIGEO_IE'] = limpiar_numerico(postulantes_df['UBIGEO_IE']).astype('Int64')
postulantes_df['EDAD'] = limpiar_numerico(postulantes_df['EDAD']).astype('Int64')
postulantes_df['PAGO'] = limpiar_numerico(postulantes_df['PAGO']).astype('Int64')

# Campos de texto (convertir a string)
text_fields = [
    'UUID', 'ESCUELA', 'MODALIDAD', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO',
    'DEPARTAMENTO_ORIGEN', 'PROVINCIA_ORIGEN', 'DISTRITO_ORIGEN',
    'DEPARTAMENTO_IE', 'PROVINCIA_IE', 'DISTRITO_IE', 'SEXO', 'IDIOMA',
    'LEE', 'ESCRIBE', 'HABLA', 'TIENE_DISCAPACIDAD', 'TIPO_COMUNIDAD', 'COMUNIDAD'
]

for field in text_fields:
    if field in postulantes_df.columns:
        postulantes_df[field] = postulantes_df[field].astype('string')

print("✅ Tipos convertidos para postulantes!")

🔄 Convirtiendo tipos de datos para postulantes...
✅ Tipos convertidos para postulantes!


In [11]:
# Verificar los tipos después de la conversión
print("📊 Tipos de datos después de la conversión:")
postulantes_df.dtypes

📊 Tipos de datos después de la conversión:


FECHA_CORTE                     Int64
UUID                   string[python]
ESCUELA                string[python]
MODALIDAD              string[python]
ANNIO_POSTULACION               Int64
UBIGEO                          Int64
DEPARTAMENTO           string[python]
PROVINCIA              string[python]
DISTRITO               string[python]
UBIGEO_ORIGEN                   Int64
DEPARTAMENTO_ORIGEN    string[python]
PROVINCIA_ORIGEN       string[python]
DISTRITO_ORIGEN        string[python]
UBIGEO_IE                       Int64
DEPARTAMENTO_IE        string[python]
PROVINCIA_IE           string[python]
DISTRITO_IE            string[python]
EDAD                            Int64
SEXO                   string[python]
PAGO                            Int64
IDIOMA                 string[python]
LEE                    string[python]
ESCRIBE                string[python]
HABLA                  string[python]
TIENE_DISCAPACIDAD     string[python]
TIPO_COMUNIDAD         string[python]
COMUNIDAD   

In [12]:
# Función para generar SQL desde DataFrame con tipos correctos
def dataframe_to_sql(df, table_name="df_table", output_file=None):
    """
    Convierte DataFrame a SQL con tipos de datos correctos
    """
    
    if output_file is None:
        output_file = f"{table_name}.sql"
    
    # Mapear tipos de pandas a PostgreSQL
    type_mapping = {
        'Int64': 'INTEGER',
        'int64': 'INTEGER', 
        'float64': 'DECIMAL(15,2)',
        'string': 'VARCHAR(500)',
        'object': 'VARCHAR(500)'
    }
    
    sql_content = []
    
    # CREATE TABLE
    sql_content.append(f"-- Tabla generada desde DataFrame")
    sql_content.append(f"DROP TABLE IF EXISTS {table_name};")
    sql_content.append(f"CREATE TABLE {table_name} (")
    
    columns = []
    for col in df.columns:
        clean_col = col.lower().replace(' ', '_').replace('-', '_')
        pandas_type = str(df[col].dtype)
        
        # Mapear tipos específicos según el diccionario de datos para postulantes
        if col in ['FECHA_CORTE', 'ANNIO_POSTULACION', 'UBIGEO', 'UBIGEO_ORIGEN', 'UBIGEO_IE', 'EDAD', 'PAGO']:
            sql_type = 'INTEGER'
        elif col == 'UUID':
            sql_type = 'VARCHAR(64)'  # UUID como string
        elif col in ['ESCUELA', 'MODALIDAD']:
            sql_type = 'VARCHAR(255)'
        elif col in ['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'DEPARTAMENTO_ORIGEN', 'PROVINCIA_ORIGEN', 'DISTRITO_ORIGEN',
                    'DEPARTAMENTO_IE', 'PROVINCIA_IE', 'DISTRITO_IE', 'SEXO', 'IDIOMA', 'TIPO_COMUNIDAD', 'COMUNIDAD']:
            sql_type = 'VARCHAR(100)'
        elif col in ['LEE', 'ESCRIBE', 'HABLA', 'TIENE_DISCAPACIDAD']:
            sql_type = 'VARCHAR(10)'  # Para SI/NO
        else:
            sql_type = type_mapping.get(pandas_type, 'VARCHAR(255)')
        
        columns.append(f"    {clean_col} {sql_type}")
    
    sql_content.append(",\n".join(columns))
    sql_content.append(");")
    sql_content.append("")
    
    # INSERT statements
    sql_content.append(f"-- Insertar datos ({len(df)} registros)")
    
    for index, row in df.iterrows():
        values = []
        for col in df.columns:
            value = row[col]
            
            if pd.isna(value) or value is None:
                values.append("NULL")
            elif isinstance(value, str) or df[col].dtype == 'string':
                # Escapar comillas simples y limpiar
                clean_value = str(value).replace("'", "''")
                values.append(f"'{clean_value}'")
            else:
                values.append(str(value))
        
        clean_columns = [col.lower().replace(' ', '_').replace('-', '_') for col in df.columns]
        columns_str = ", ".join(clean_columns)
        values_str = ", ".join(values)
        
        sql_content.append(f"INSERT INTO {table_name} ({columns_str}) VALUES ({values_str});")
        
        # Progreso cada 1000 registros
        if (index + 1) % 1000 == 0:
            print(f"📝 Procesados {index + 1} registros...")
    
    # Escribir archivo
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sql_content))
    
    print(f"✅ Archivo SQL generado: {output_file}")
    print(f"📊 Registros: {len(df)}")
    print(f"📋 Columnas: {len(df.columns)}")
    
    return output_file

# Generar el SQL para postulantes
sql_file = dataframe_to_sql(postulantes_df, "postulantes", "postulantes_clean.sql")

📝 Procesados 1000 registros...
📝 Procesados 2000 registros...
📝 Procesados 3000 registros...
📝 Procesados 4000 registros...
✅ Archivo SQL generado: postulantes_clean.sql
📊 Registros: 4928
📋 Columnas: 27
✅ Archivo SQL generado: postulantes_clean.sql
📊 Registros: 4928
📋 Columnas: 27
